# Profitable localities for Pizza shop in Bangalore

In [2]:
import pandas as pd
import numpy as np
import requests
import folium
import seaborn as sns
import statsmodels.formula.api as smf

## Data

Bangalore. Bangalore political map in `geoJSON` format was obtained from [__Data{Meet}__](https://raw.githubusercontent.com/datameet/Municipal_Spatial_Data/master/Bangalore/BBMP.GeoJSON) community. This data has boundary information of 198 wards in Bangalore along with population and area of each ward.

Using this ward information, all the dinner restaurants in these wards were collected from __[Foursquare](https://developer.foursquare.com/)__ using their APIs.

Below is the top 5 rows (head of the data) of Bangalore ward information.

In [3]:
bang_wards = pd.read_csv('C:/Users/suhas/OneDrive/Battle of Neighborhood/Pizza/bang_df.csv')
bang_wards.head()

,name,lat,lon,pop,area,rest_count,pizza_count
0,Chowdeswari Ward,13.121709,77.580422,19626,7.06,0,0
1,Atturu,13.102805,77.560038,24020,10.15,1,0
2,Yelahanka Satellite Town,13.090987,77.583925,25782,4.90,13,5
3,Vijnanapura,13.006063,77.669565,35087,2.05,0,0
4,Basavanapura,13.016847,77.715456,22012,6.28,0,0


Using Foursquare APIs, data of all the dinner restaurants in Bangalore is obtained as shown below.

In [4]:
CLIENT_ID = 'MZY14SRHWBY5RH3Q1CWTIN2GXVPRPXB1TYDIJIJTGR1SRESP'
CLIENT_SECRET = 'YR1UZK5C1FNS5B5XNVITPIQALNGVGC0KB2TGS1VSQR53A0LP'

#CLIENT_ID = 'QRQV2F25X345H0Y244BQTDNNQZT3SNBA02VYPDIWIIXGYMPW' # your Foursquare ID
#CLIENT_SECRET = '2500R2BVXUKLIZWX5ONDVS5I1GAHSYPLNIT0U5COA1J2RS03' # your Foursquare Secret
VERSION = '20190326'


def getNearbyVenuesNew(names, latitudes, longitudes, radius=500, query=None):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        try:  
            # create the API request URL
            if(query!=None):
                url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}'.format(
                    CLIENT_ID, 
                    CLIENT_SECRET, 
                    VERSION, 
                    lat, 
                    lng, 
                    radius, 
                    100,
                    query)
            else:
                url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                    CLIENT_ID, 
                    CLIENT_SECRET, 
                    VERSION, 
                    lat, 
                    lng, 
                    radius, 
                    100)
           # print(url)
            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']
            # return only relevant information for each nearby venue
            venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
                #print(venues_list)
                
        except:
            pass
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
    'Neighborhood Latitude', 
    'Neighborhood Longitude', 
    'Venue', 
    'Venue Latitude', 
    'Venue Longitude', 
    'Venue Category']
    return(nearby_venues)

def drange(x, y, jump):
    while x <= y:
        yield float(x)
        x += jump

Below is the head of all the restaurants in Bangalore

In [5]:
bang_dinner = getNearbyVenuesNew(names=bang_wards['name'],
                                   latitudes=bang_wards['lat'],
                                   longitudes=bang_wards['lon'],
                                   radius = 5000,
                                   query='dinner'
                                  )
bang_dinner = bang_dinner.drop_duplicates(subset=['Venue','Venue Latitude','Venue Longitude'],keep='first')
bang_dinner.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Chowdeswari Ward,13.121709,77.580422,Cafe Coffee Day,13.103561,77.584409,Café
1,Chowdeswari Ward,13.121709,77.580422,Cafe Coffee Day,13.099390,77.588282,Café
2,Chowdeswari Ward,13.121709,77.580422,Cafe Coffee Day,13.094997,77.597301,Café
3,Chowdeswari Ward,13.121709,77.580422,Cafe Potenza,13.121925,77.623036,Café
4,Chowdeswari Ward,13.121709,77.580422,Subway,13.099440,77.589355,American Restaurant


All these restaurants are mapped using Folium.

In [8]:
latitude = 12.979682
longitude = 77.610761

bang_map = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lon, poi in zip(bang_dinner['Venue Latitude'], bang_dinner['Venue Longitude'], bang_dinner['Venue']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.Circle(
        [lat, lon],
        popup=label,
        radius=2,
        fill=True,
        fill_opacity=0.9).add_to(bang_map)
    
bang_map

Similarly, all the pizza restaurants details have been retreived from Foursquare. Below is the head of data.

In [6]:
bang_pizza = getNearbyVenuesNew(names=bang_wards['name'],
                                   latitudes=bang_wards['lat'],
                                   longitudes=bang_wards['lon'],
                                   radius = 5000,
                                   query='pizza'
                                  )
bang_pizza = bang_pizza.drop_duplicates(subset=['Venue','Venue Latitude','Venue Longitude'],keep='first')
bang_pizza.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Chowdeswari Ward,13.121709,77.580422,Grub,13.092928,77.587779,Pizza Place
1,Chowdeswari Ward,13.121709,77.580422,Domino's Pizza,13.099618,77.589035,Pizza Place
2,Chowdeswari Ward,13.121709,77.580422,Pizza Hut Delivery,13.099336,77.589441,Pizza Place
3,Atturu,13.102805,77.560038,Domino's,13.078761,77.559073,Pizza Place
6,Atturu,13.102805,77.560038,Domino's Pizza,13.081000,77.556000,Pizza Place


Using Folium, pizza restaurants are mapped in different color.

In [10]:
latitude = 12.979682
longitude = 77.610761

bang_map = folium.Map(location=[latitude, longitude], zoom_start=11)

    
for lat, lon, poi in zip(bang_pizza['Venue Latitude'], bang_pizza['Venue Longitude'], bang_pizza['Venue']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.Circle(
        [lat, lon],
        color='red',
        popup=label,
        radius=2,
        fill=True,
        fill_opacity=0.9).add_to(bang_map)
    
bang_map